# Corynocarpus rupestris
## ONT Basecalling

### Run statistics (Before Basecalling):
- LSK114 kit

N50 = 15.3kb
Reads = 1.47M
Bases = 10.38 Gbases
Data = 114 GB
Coverage = ~15X

In [1]:
WKDIR=/workspace/hraijc/Karaka/Crupestris/01_basecalling
READDIR=/input/genomic/plant/Corynocarpus/rupestris/Crup_ONT1/Crup_ONT1/20240926_1423_MN19482_FAZ32631_9de6a161/pod5
DORADO=/workspace/hraijc/git_clones/dorado-0.7.3-linux-x64/bin/dorado
PREFIX=Crup_ONT1

In [2]:
cd $WKDIR
mkdir -p ${WKDIR}/log

In [3]:
$DORADO -v

[2024-09-29 12:07:19.288] [info] Running: "-v"
0.7.3+6e6c45c


In [5]:
$DORADO basecaller -h

[2024-09-24 10:58:53.234] [info] Running: "basecaller" "-h"
Usage: dorado [-h] [--device VAR] [--read-ids VAR] [--resume-from VAR] [--max-reads VAR] [--min-qscore VAR] [--batchsize VAR] [--chunksize VAR] [--overlap VAR] [--recursive] [--modified-bases VAR...] [--modified-bases-models VAR] [--modified-bases-threshold VAR] [--emit-fastq] [--emit-sam] [--emit-moves] [--reference VAR] [--kit-name VAR] [--barcode-both-ends] [--no-trim] [--trim VAR] [--sample-sheet VAR] [--barcode-arrangement VAR] [--barcode-sequences VAR] [--primer-sequences VAR] [--estimate-poly-a] [--poly-a-config VAR] [-k VAR] [-w VAR] [-I VAR] [--secondary VAR] [-N VAR] [-Y] [--bandwidth VAR] [--junc-bed VAR] [--mm2-preset VAR] model data

Positional arguments:
  model                     	model selection {fast,hac,sup}@v{version} for automatic model selection including modbases, or path to existing model directory 
  data                      	the data directory or file (POD5/FAST5 format). 

Optional arguments:
  -h, 

In [4]:
#DORADO duplex
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=160G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=47:00:00
#SBATCH --gres=gpu:1

module load samtools/1.20
cd ${WKDIR}

#Basecalling
${DORADO} duplex sup ${READDIR} > ${PREFIX}_duplex.bam

# Get QC report of runs. 
~/.local/bin/NanoPlot -t 8 --ubam ${PREFIX}_duplex.bam -o ${WKDIR}/${PREFIX}_summary-plots

#Bam -> Fastq
samtools fastq -@ 8 -T '*' ${PREFIX}_duplex.bam > ${PREFIX}_all.fastq
/workspace/hraijc/git_clones/seqkit/seqkit seq ${PREFIX}_all.fastq -j 8 -Q 15 -m 1000 > ${PREFIX}_q15L1k.fastq

#Stats
/workspace/hraijc/git_clones/seqkit/seqkit stats *.fastq

#zip outputs
gzip *.fastq


EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 7723416


In [3]:
du -sh ${WKDIR}/*.bam


12G	/workspace/hraijc/Karaka/Crupestris/01_basecalling/Crup_ONT1_duplex.bam


In [4]:
#DoradoDuplex ATD
seff 7723416

Job ID: 7723416
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 8
CPU Utilized: 10:42:43
CPU Efficiency: 14.27% of 3-03:04:56 core-walltime
Job Wall-clock time: 09:23:07
Memory Utilized: 160.00 GB
Memory Efficiency: 100.00% of 160.00 GB


In [5]:
cat ${WKDIR}/log/7723416.out

file                    format  type   num_seqs         sum_len  min_len  avg_len    max_len
Crup_ONT1_all.fastq     FASTQ   DNA   1,698,879  12,710,424,334        5  7,481.7  1,787,052
Crup_ONT1_q15L1k.fastq  FASTQ   DNA   1,154,494  10,240,802,276    1,000  8,870.4    263,124


In [6]:
ls $WKDIR

Crup_ONT1_all.fastq.gz	Crup_ONT1_duplex.bam.bai   Crup_ONT1_summary-plots
Crup_ONT1_duplex.bam	Crup_ONT1_q15L1k.fastq.gz  log
